### Default Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup

%matplotlib inline

### Modeling imports

## Doing the modeling!

In [5]:
merged = pd.read_csv('./data/merged_v3.csv')
merged.head()

,year,player,listed_pos,age,tm_name,g,gs,mp,per,ts%,...,blk,tov,pf,pts,year-player,award_tm,award_pos,starter,reserve,all_star
0,1989.0,kareem abdul-jabbar,C,41.0,LAL,74.0,74.0,1695.0,12.9,0.511,...,85.0,95.0,196.0,748.0,1989.0kareem abdul-jabbar,0,C,0.0,1.0,1.0
1,1989.0,mark acres,C,26.0,BOS,62.0,0.0,632.0,8.2,0.507,...,6.0,23.0,94.0,137.0,1989.0mark acres,0,C,0.0,0.0,0.0
2,1989.0,michael adams,PG,26.0,DEN,77.0,77.0,2787.0,17.5,0.567,...,11.0,180.0,149.0,1424.0,1989.0michael adams,0,PG,0.0,0.0,0.0
3,1989.0,mark aguirre,SF,29.0,TOT,80.0,76.0,2597.0,16.4,0.524,...,36.0,208.0,229.0,1511.0,1989.0mark aguirre,0,SF,0.0,0.0,0.0
4,1989.0,danny ainge,PG,29.0,TOT,73.0,54.0,2377.0,18.0,0.554,...,8.0,145.0,186.0,1281.0,1989.0danny ainge,0,PG,0.0,0.0,0.0


In [6]:
merged['target'] = merged['award_tm'].map(lambda x: 1 if x > 0 else 0)

**Notes**
- Need to actually hold out a couple years fully for unseen data. So we'll have:
    - Train data (20)
    - Test data (6)
    - Holdout data (2)
- Look up `Categorical Encoder`

In [8]:
## gonna drop 1999 because of no all star game...
model_df = merged[merged['year'] != 1999]


train_yrs = [1989, 1990, 1991,
             1994, 1995,
             1997, 1998, 2000,
             2002, 2003, 2004,
             2006, 2007, 2008,
             2010, 2011, 2012,
             2014, 2015, 2016]
             
test_yrs = [1992,
            1996,
            2001,
            2005,
            2013,
            2017]

hold_yrs = [1993, 2009]

X_train = model_df.drop(['target', 'year-player', 'player', 'award_tm'], axis=1).loc[model_df['year'].isin(train_yrs)]
X_test = model_df.drop(['target', 'year-player', 'player', 'award_tm'], axis=1).loc[model_df['year'].isin(test_yrs)]

y_train = model_df.loc[model_df['year'].isin(train_yrs), 'target']
y_test = model_df.loc[model_df['year'].isin(test_yrs), 'target']

hold_df = model_df.loc[model_df['year'].isin(hold_yrs)]

print('Train shapes',X_train.shape, y_train.shape)
print('Test shapes',X_test.shape, y_test.shape)
print('Hold shape',hold_df.shape)

Train shapes (8736, 53) (8736,)
Test shapes (2671, 53) (2671,)
Hold shape (833, 57)


### Start here!!
- model TTS starting point

In [9]:
## saving progress
# X_train.to_csv('./data/X_train.csv')
# X_test.to_csv('./data/X_test.csv')
# y_train.to_csv('./data/y_train.csv')
# y_test.to_csv('./data/y_test.csv')

# X_train = pd.read_csv('./data/X_train.csv', index_col=0)
# X_test = pd.read_csv('./data/X_test.csv', index_col=0)
# y_train = pd.read_csv('./data/y_train.csv', index_col=0)
# y_test = pd.read_csv('./data/y_test.csv', index_col=0)

## Model time!!
1. Set up pipelines!